##### Token Initialization

In [1]:
import pandas as pd
df=pd.read_csv('PoetryFoundationData.csv')
df.columns

col1=df['Poem']
entry=""
for i in range(0,21):
    for j in col1[i]:
        entry=entry+j

import re
text=re.split(r'(\s)', entry)
#Removing spaces

new_list=[]
for i in text:
    new=i.strip()
    if new!="":
        new_list.append(new)

#print(new_list)
new_list.sort()

##### Token ID

In [2]:
id={}
for i in range(len(new_list)):
    id.update({new_list[i]:i})

#print(id)

##### Tokenizer

In [3]:
class tokenizer:
    def __init__(self, dic):
        self.dic=dic

        dic1={}
        for i,j in dic.items():
            dic1.update({j:i})

        self.inverse_dic=dic1
    
    def encode(self, txt):
        text=re.split(r'([,:?;]|\s)', txt)
        id=[]
        for i in text:
            j=i.strip()
            if j!="":
                id.append(self.dic[j])
        
        return id
    
    def decode(self, id):
        st=""
        for i in range(len(id)):
            if id==len(id)-1:
                st=st+self.inverse_dic[id[i]]
            else:
                st=st+self.inverse_dic[id[i]]+" "
        
        return st

token1=tokenizer(id)
entry=token1.encode("It's a big Dog")
final=token1.decode(entry)
print(final)


It's a big Dog 


##### Byte Pair Encoding

In [4]:
import tiktoken
tokenizer1=tiktoken.get_encoding("gpt2")
codes=tokenizer1.encode("I love BMW")
text=tokenizer1.decode(codes)
print(text)

I love BMW


##### Sliding Window Algorithm

In [5]:
text_to_encode=col1[5]
limit=32
actual=text_to_encode[0:limit]

X=[]
Y=[]
for i in actual.split(" "):
    Y.append(i.strip())
    X.append(i.strip())


Y.remove(Y[0])
X.remove(X[-1])

print(actual+"\n")

X_id=[]
Y_id=[]
for i in X:
    X_id.append(id[i])

for i in Y:
    Y_id.append(id[i])

import torch

x_tensor=[]
y_tensor=[]

for i in range(0, len(X_id), 2):
    li=[]
    li.append(X_id[i])
    li.append(X_id[i+1])

    li1=[]
    li1.append(Y_id[i])
    li1.append(Y_id[i+1])

    x_tensor.append(li)
    y_tensor.append(li1)

x_tensor=torch.tensor(x_tensor)
y_tensor=torch.tensor(y_tensor)
print(x_tensor)
print(y_tensor)


You are the start of the week

tensor([[ 342,  582],
        [2694, 2396],
        [1956, 2694]])
tensor([[ 582, 2694],
        [2396, 1956],
        [2694, 2975]])


##### Embedding

In [6]:
embedding=torch.nn.Embedding(len(new_list), 3)
#print(embedding.weight)
embebbed_layer=embedding(x_tensor)
print(embebbed_layer)

tensor([[[ 1.3977, -0.9286, -1.3481],
         [-1.3189, -0.8629, -2.0889]],

        [[ 0.5474,  0.0208,  1.4589],
         [-0.7708,  0.5540,  0.6492]],

        [[ 1.6639,  0.0560,  0.2612],
         [ 0.5474,  0.0208,  1.4589]]], grad_fn=<EmbeddingBackward0>)


##### Attention Scores

In [7]:
new_dim=[]
for i in embebbed_layer:
    for j in i:
        new_dim.append(j.tolist())   

new_dim=torch.tensor(new_dim)

attention=torch.empty(6,6)


for i,j in enumerate(new_dim):
    for k,l in enumerate(new_dim):
        attention[i,k]=torch.dot(j,l)

attention=torch.softmax(attention, dim=1)
print(attention)

context=attention@new_dim
print("\n--------------Context Vector---------------\n")
print(context)

tensor([[8.8472e-01, 5.0704e-02, 2.5375e-03, 7.2986e-04, 5.8772e-02, 2.5375e-03],
        [6.2161e-03, 9.9321e-01, 2.3893e-05, 4.6567e-04, 6.4876e-05, 2.3893e-05],
        [1.0403e-02, 7.9895e-04, 3.9998e-01, 6.0317e-02, 1.2852e-01, 3.9998e-01],
        [1.0554e-02, 5.4925e-02, 2.1275e-01, 4.6686e-01, 4.2159e-02, 2.1275e-01],
        [2.1594e-01, 1.9443e-03, 1.1518e-01, 1.0712e-02, 5.4104e-01, 1.1518e-01],
        [1.0403e-02, 7.9895e-04, 3.9998e-01, 6.0317e-02, 1.2852e-01, 3.9998e-01]])

--------------Context Vector---------------

tensor([[ 1.2697, -0.8615, -1.2754],
        [-1.3015, -0.8625, -2.0827],
        [ 0.6187,  0.0469,  1.2241],
        [-0.1145,  0.2127,  0.8059],
        [ 1.3174, -0.1612,  0.1892],
        [ 0.6187,  0.0469,  1.2241]])


##### Attention Mechanism With Trainable Weights

In [8]:
# Input embebbed layer
input=embebbed_layer.reshape(-1, embebbed_layer.shape[2])


torch.manual_seed(150)
query=torch.nn.Parameter(torch.rand(3,2), requires_grad=False)
key=torch.nn.Parameter(torch.rand(3,2), requires_grad=False)
value=torch.nn.Parameter(torch.rand(3,2), requires_grad=False)

query_vector=input@query
key_vector=input@key
value_vector=input@value 

# Attention Score Calculation
attention_scores=query_vector @ key_vector.T

#Scaling and Normalization
key_shape=key_vector.shape[-1]
attention_scores_final=torch.softmax((attention_scores/(key_shape**0.5)), dim=1)
print("-----------ATTENTION SCORES-------------\n")
print(attention_scores_final)
#Context Vector
context_vector=attention_scores_final @ value_vector
print("-----------CONTEXT VECTOR-------------\n")
print(context_vector)


-----------ATTENTION SCORES-------------

tensor([[0.1902, 0.2599, 0.1337, 0.1485, 0.1340, 0.1337],
        [0.0555, 0.9390, 0.0010, 0.0023, 0.0013, 0.0010],
        [0.0317, 0.0078, 0.2761, 0.1798, 0.2284, 0.2761],
        [0.1584, 0.1391, 0.1772, 0.1692, 0.1789, 0.1772],
        [0.0335, 0.0078, 0.2746, 0.1737, 0.2359, 0.2746],
        [0.0317, 0.0078, 0.2761, 0.1798, 0.2284, 0.2761]],
       grad_fn=<SoftmaxBackward0>)
-----------CONTEXT VECTOR-------------

tensor([[-0.1446, -0.0892],
        [-1.6522, -1.6847],
        [ 0.5177,  0.5820],
        [ 0.1583,  0.2235],
        [ 0.5267,  0.5932],
        [ 0.5177,  0.5820]], grad_fn=<MmBackward0>)


##### Causal Attention Mechanism

In [9]:
# Using Previous Attention Vector

masked_matrix=torch.tril(torch.ones(6,6))
new_attention=attention_scores_final * masked_matrix
print("-----------ATTENTION SCORES-------------\n")

#Normalization
row=new_attention.sum(dim=1, keepdim=True)
masked_attention=new_attention/row

#Fixing Normalization Problem
fix=torch.triu(torch.ones(6,6), diagonal=1)
fixed_attention=attention_scores_final.masked_fill(fix.bool(), -torch.inf)

#Softmax with scaling
attention_score=torch.softmax(fixed_attention/(key_shape**0.5), dim=1)

#Dropout Phase
dropout=torch.nn.Dropout(0.4)
attention_score_1=dropout(attention_score)

print(attention_score_1)
print("-----------CONTEXT VECTOR-------------\n")
context_vector_causal=attention_score_1 @ value_vector
print(context_vector_causal)



-----------ATTENTION SCORES-------------

tensor([[1.6667, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5812, 1.0855, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5255, 0.5166, 0.6246, 0.0000, 0.0000, 0.0000],
        [0.4159, 0.0000, 0.0000, 0.4191, 0.0000, 0.0000],
        [0.3072, 0.0000, 0.0000, 0.0000, 0.3544, 0.0000],
        [0.2518, 0.2475, 0.0000, 0.2795, 0.0000, 0.2993]],
       grad_fn=<MulBackward0>)
-----------CONTEXT VECTOR-------------

tensor([[ 0.3135,  0.7107],
        [-1.8154, -1.7297],
        [-0.5410, -0.3885],
        [ 0.0499,  0.0904],
        [ 0.5131,  0.6420],
        [-0.2781, -0.2441]], grad_fn=<MmBackward0>)


In [10]:
import torch.nn as nn

class Head(nn.Module):
    def __init__(self, dropout, dim_in, dim_out, context_size, bias=False):

        super().__init__()
        self.out=dim_out
        self.query=nn.Linear(dim_in,dim_out, bias=bias)
        self.key=nn.Linear(dim_in,dim_out, bias=bias)
        self.value=nn.Linear(dim_in,dim_out, bias=bias)
        self.dropout=nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_size, context_size)), diagonal=1)
    
    def forward(self, x):
        query_vector=self.query(x)
        key_vector=self.key(x)
        value_vector=self.value(x)
        outside, border, inside=x.shape

        attention_scores=query_vector @ key_vector.transpose(1,2)
        
        scores = attention_scores.masked_fill(self.mask.bool()[:border, :border], -torch.inf)
        attention_1=torch.softmax(scores/(key_vector.shape[-1]**0.5), dim=-1)

        attention_score=self.dropout(attention_1)
        context_vector=attention_score @ value_vector

        return context_vector

In [11]:
class MultiHead(nn.Module):
    def __init__(self, d_in, d_out, bias, dropout, context_length, iterations):
        super().__init__()
        self.head=nn.ModuleList()
        for i in range(iterations):
            self.head.append(Head(d_in, d_out, dropout, context_length, bias=False))
    
    def forward(self, x):
        
        output=[]
        for h in self.head:
            output.append(h(x))
        
        final=torch.cat(output, dim=-1)
        return final


### Normalisation

In [26]:

test=embebbed_layer[0,1]
mean=test.mean(dim=-1, keepdim=True)
var=test.var(dim=-1, keepdim=True)
test=(test-mean)/torch.sqrt(var)

v1=test.mean(dim=-1, keepdim=True)
v2=test.var(dim=-1, keepdim=True)

print(v1)
print(v2)
print("\n")
print("-------Normalised Matrix-------")
print(test)

tensor([-1.1921e-07], grad_fn=<MeanBackward1>)
tensor([1.0000], grad_fn=<VarBackward0>)


-------Normalised Matrix-------
tensor([ 0.1689,  0.9048, -1.0737], grad_fn=<DivBackward0>)
